<a href="https://colab.research.google.com/github/MemoGhazi/NLP/blob/main/Summarization_Bart_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
ARTICLE1 =  """A poor man had twelve children and was forced to work night and
day to give them even bread.  When therefore the thirteenth
came into the world, he knew not what to do in his trouble,
but ran out into the great highway, and resolved to ask the
first person whom he met to be godfather.  The first to meet
him was the good God who already knew what filled his heart,
and said to him, poor man, I pity you.  I will hold your child
at its christening, and will take charge of it and make it
happy on earth.  The man said, who are you.  I am God.  Then
I do not desire to have you for a godfather, said the man, you give
to the rich, and leave the poor to hunger.  Thus spoke the man,
for he did not know how wisely God apportions riches and
poverty.  He turned therefore away from the Lord, and went
farther.  Then the devil came to him and said, what do you seek.
If you will take me as a godfather for your child, I will give him
gold in plenty and all the joys of the world as well.  The man
asked, who are you.  I am the devil.  Then I do not desire to have
you for godfather, said the man, you deceive men and lead them
astray.  He went onwards, and then came death striding up to
him with withered legs, and said, take me as godfather.  The
man asked, who are you.  I am death, and I make all equal.  Then
said the man,
you are the right one, you take the rich as well as the poor,
without distinction, you shall be godfather.  Death answered,
I will make your child rich and famous, for he who has me for a
friend can lack nothing.  The man said, next sunday is the
christening, be there at the right time.  Death appeared as he
had promised, and stood godfather quite in the usual way.
When the boy had grown up, his godfather one day appeared
and bade him go with him. He led him forth into a forest, and
showed him a herb which grew there, and said, now you shall
receive your godfather's present.  I make you a celebrated
physician.  When you are called to a patient, I will always appear
to you.  If I stand by the head of the sick man, you may say with
confidence that you will make him well again, and if you give
him of this herb he will recover, but if I stand by the patient's
feet, he is mine, and you must say that all remedies are in
vain, and that no physician in the world could save him.  But
beware of using the herb against my will, or it might fare
ill with you.
It was not long before the youth was the most famous physician
in the whole world.  He had only to look at the patient and he
knew his condition at once, whether he would recover, or must
needs die.  So they said of him, and from far and wide people
came to him, sent for him when they had anyone ill, and gave him
so much money that he soon became a rich man.  Now it so befell
that the king became ill, and the physician was summoned, and
was to say if recovery were possible.  But when he came to the bed,
death was standing by the feet of the sick man, and the herb
did not grow which could save him.  If I could but cheat death for
once, thought the physician, he is sure to take it ill if I do
but, as I am his godson, he will shut one eye, I will risk it.  He
therefore took up the sick man, and laid him the other way, so
that now death was standing by his head.  Then he gave the king
some of the herbs, and he recovered and grew healthy again.
But death came to the physician, looking very black and
angry, threatened him with his finger, and said, you have betrayed
me, this time I will pardon
it, as you are my godson, but if you venture it again, it
will cost you your neck, for I will take you yourself away
with me.
Soon afterwards the king's daughter fell into a severe illness.
She was his only child, and he wept day and night, so that he
began to lose the sight of his eyes, and he caused it to be
made known that whosoever rescued her from death should be her
husband and inherit the crown.  When the physician came to the
sick girl's bed, he saw death by her feet.  He ought to have
remembered the warning given by his godfather, but he was so
infatuated by the great beauty of the king's daughter, and the
happiness of becoming her husband, that he flung all thought to
the winds.  He did not see that death was casting angry glances
on him, that he was raising his hand in the air, and threatening
him with his withered fist.  He raised up the sick girl,
and placed her head where her feet had lain.  Then he gave
her some of the herb, and instantly her cheeks flushed red,
and life stirred afresh in her.
When death saw that for a second time his own property had been
misused, he walked up to the physician with long strides, and
said, all is over with you, and now the lot falls on you, and
seized him so firmly with his ice-cold hand, that he could not
resist, and led him into a cave below the earth.  There he
saw how thousands and thousands of candles were burning in
countless rows, some large, some medium-sized, others small.
Every instant some were extinguished, and others again burnt up,
so that the flames seemed to leap hither and thither in
perpetual change.  See, said death, these are the lights of
men's lives.  The large ones belong to children, the medium-sized
ones to married people in their prime, the little ones belong
to old people, but children and young folks likewise have
often only a tiny candle.  Show me the light of my life, said
the physician, and he thought that it would be still very
tall.  Death pointed to a little end which was just threatening
to go out, and said, behold, it is there.  Ah, dear godfather,
said the horrified physician, light a new one for me, do it for
love of me, that I may enjoy my life, be king, and the husband of
the king's beautiful daughter.  I cannot, answered death,
one must go out
before a new one is lighted.  Then place the old one on a new
one, that will go on burning at once when the old one has come
to an end, pleaded the physician.  Death behaved as if he were
going to fulfill his wish, and took hold of a tall new candle,
but as he desired to revenge himself, he purposely made a mistake
in fixing it, and the little piece fell down and was extinguished.
Immediately the physician fell on the ground, and now he himself
was in the hands of death."""

In [47]:
# tokenize without truncation
inputs_no_trunc = tokenizer(ARTICLE1, max_length=None, return_tensors='pt', truncation=False)

# get batches of tokens corresponding to the exact model_max_length
chunk_start = 0
chunk_end = tokenizer.model_max_length  # == 1024 for Bart
inputs_batch_lst = []
while chunk_start <= len(inputs_no_trunc['input_ids'][0]):
    inputs_batch = inputs_no_trunc['input_ids'][0][chunk_start:chunk_end]  # get batch of n tokens
    inputs_batch = torch.unsqueeze(inputs_batch, 0)
    inputs_batch_lst.append(inputs_batch)
    chunk_start += tokenizer.model_max_length  # == 1024 for Bart
    chunk_end += tokenizer.model_max_length  # == 1024 for Bart

# generate a summary on each batch
summary_ids_lst = [model.generate(inputs, num_beams=4, max_length=100, early_stopping=True) for inputs in inputs_batch_lst]



In [45]:
# decode the output and join into one string with one paragraph per summary batch
summary_batch_lst = []
for summary_id in summary_ids_lst:
    summary_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_id]
    summary_batch_lst.append(summary_batch[0])
summary_all = '\n'.join(summary_batch_lst)

print(summary_all)

A poor man had twelve children and was forced to work night and day to give them even bread. He resolved to ask the first person whom he met to be godfather. The good God said to him, poor man, I pity you. I will hold your child at its christening. The man said, who are you.  I am God. Then the devil came to him and said, what do you seek. If you will take me as a godfather for your
The physician was so infatuated by the great beauty of the king's daughter that he flung all thought to the winds. He raised up the sick girl, placed her head where her feet had lain. Then he gave her some of the herb, and instantly her cheeks flushed red, and life stirred afresh in her.
